In [1]:
import sys
import os

from IPython.display import display

sys.path.append(os.path.join(os.path.abspath(''), '..', '..'))

from tools.profiler import generate_schedule, parse_result, print_result, draw_barh

dlrm_perf_schedule = {
    # interested event name
    'BottomMLP.fc1': [
        'fused_fully_connected.fprop',
#        'fused_fully_connected.fprop.cublasGemmEx',
#        'fused_fully_connected.fprop.add_bias_and_re_kernel',
        'fused_fully_connected.bprop',
#        'fused_fully_connected.bprop.initialize_array',
        'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
        'fused_fully_connected.bprop.convert_array',
        'fused_fully_connected.bprop.cublasGemmEx_1',
        'fused_fully_connected.bprop.cublasGemmEx_2'
    ],
    'BottomMLP.fc2': [
        'fused_fully_connected.fprop',
        'fused_fully_connected.fprop.cublasGemmEx',
        'fused_fully_connected.fprop.add_bias_and_re_kernel',
        'fused_fully_connected.bprop',
        'fused_fully_connected.bprop.initialize_array',
#        'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
        'fused_fully_connected.bprop.convert_array',
        'fused_fully_connected.bprop.cublasGemmEx_1',
#        'fused_fully_connected.bprop.cublasGemmEx_2'
    ],
    'BottomMLP.fc3': [
        'fused_fully_connected.fprop',
        'fused_fully_connected.fprop.cublasGemmEx',
        'fused_fully_connected.fprop.add_bias_and_re_kernel',
#        'fused_fully_connected.bprop',
        'fused_fully_connected.bprop.initialize_array',
        'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
#        'fused_fully_connected.bprop.convert_array',
        'fused_fully_connected.bprop.cublasGemmEx_1',
        'fused_fully_connected.bprop.cublasGemmEx_2'
    ],
    'sparse_embedding1': [
        'localized_slot_sparse_embedding_one_hot.forward.mapping_and_fuse'
    ],
    'interaction1': [

    ],
    'TopMLP.fc4': [
        'fused_fully_connected.fprop',
        'fused_fully_connected.fprop.cublasGemmEx',
#        'fused_fully_connected.fprop.add_bias_and_re_kernel',
        'fused_fully_connected.bprop',
        'fused_fully_connected.bprop.initialize_array',
        'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
#        'fused_fully_connected.bprop.convert_array',
        'fused_fully_connected.bprop.cublasGemmEx_1',
        'fused_fully_connected.bprop.cublasGemmEx_2'
    ],
    'TopMLP.fc5': [
        'fused_fully_connected.fprop',
#        'fused_fully_connected.fprop.cublasGemmEx',
        'fused_fully_connected.fprop.add_bias_and_re_kernel',
#        'fused_fully_connected.bprop',
        'fused_fully_connected.bprop.initialize_array',
#        'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
        'fused_fully_connected.bprop.convert_array',
#        'fused_fully_connected.bprop.cublasGemmEx_1',
        'fused_fully_connected.bprop.cublasGemmEx_2'
    ],
    'TopMLP.fc6': [
        'fused_fully_connected.fprop',
        'fused_fully_connected.fprop.cublasGemmEx',
        'fused_fully_connected.fprop.add_bias_and_re_kernel',
        'fused_fully_connected.bprop',
        'fused_fully_connected.bprop.initialize_array',
        'fused_fully_connected.bprop.reverse_add_bias_and_re_kernel',
        'fused_fully_connected.bprop.convert_array',
        'fused_fully_connected.bprop.cublasGemmEx_1',
        'fused_fully_connected.bprop.cublasGemmEx_2'
    ],
    'TopMLP.fc7': [
    ]
}

In [5]:
generate_schedule(dlrm_perf_schedule, repeat_for_each_event=50)
#some how run the hugectr --train, not added yet
result = parse_result()
#print(result)
print_result(result)
#draw_barh(result)

{
  "iter_time_ms": 2.015170693508414,
  "layers": {
    "sparse_embedding1": {
      "device_0": {
        "stream_0": [
          {
            "name": "localized_slot_sparse_embedding_one_hot.forward.mapping_and_fuse",
            "start_index": 0,
            "end_index": 1,
            "avg_measured_times_ms": 0.16959399978319803
          }
        ]
      },
      "device_1": {
        "stream_1": [
          {
            "name": "localized_slot_sparse_embedding_one_hot.forward.mapping_and_fuse",
            "start_index": 2,
            "end_index": 3,
            "avg_measured_times_ms": 0.13005653454197777
          }
        ]
      },
      "device_2": {
        "stream_2": [
          {
            "name": "localized_slot_sparse_embedding_one_hot.forward.mapping_and_fuse",
            "start_index": 4,
            "end_index": 5,
            "avg_measured_times_ms": 0.10413800024737914
          }
        ]
      },
      "device_3": {
        "stream_3": [
          {
  